In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
# Get current working directory
os.getcwd()

'/Users/mgibbs/Documents/ML1-Final-Project'

In [3]:
# Get data
aff = pd.read_csv('data/cleaned_data/clean_affiliations.csv', index_col=0)
coauth = pd.read_csv('data/cleaned_data/clean_coauthors.csv', index_col=0)
jour = pd.read_csv('data/cleaned_data/clean_journals.csv', index_col=0)
scopus = pd.read_csv('data/cleaned_data/clean_scopus.csv', index_col=0)
ninds = pd.read_csv('data/cleaned_data/clean_ninds_scopus_idx.csv', index_col=0)

/Users/mgibbs/anaconda3/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)
/Users/mgibbs/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/mgibbs/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1,3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## ninds

In [4]:
# Obtain ninds that is found in aff by inner joining ninds and aff on scopus_idx, state, and city 
ninds = pd.merge(ninds, aff, left_on=['scopus_idx', 'ninds_aff_state', 'ninds_aff_city'], 
                 right_on=['scopus_idx', 'aff_state', 'aff_city'], how='inner')

In [5]:
# Drop aff columns
ninds = ninds.drop(columns=ninds.columns[14:])

In [6]:
# Drop duplicate rows
ninds = ninds.drop_duplicates()

In [7]:
# Drop grant_application_id 
ninds = ninds.drop(columns='grant_application_id')

In [8]:
# One hot encode grant_type
ninds = pd.concat([ninds, pd.get_dummies(ninds['grant_type'], prefix='ninds_grant_type')], 
                  axis=1).drop(['grant_type'], axis=1)

In [9]:
# One hot encode fiscal_year
ninds = pd.concat([ninds, pd.get_dummies(ninds['fiscal_year'], prefix='ninds_fiscal_year')], 
                  axis=1).drop(['fiscal_year'], axis=1)

In [10]:
# Rename support_year
ninds = ninds.rename(columns={'support_year': 'ninds_support_year'})

In [11]:
# Get project_duration_days by subtracting project_start_date from project_end_date 
ninds['project_end_date'] = pd.to_datetime(ninds['project_end_date'])
ninds['project_start_date'] = pd.to_datetime(ninds['project_start_date'])
ninds['ninds_project_duration_days'] = (ninds['project_end_date'] - ninds['project_start_date']).dt.days
ninds = ninds.drop(columns=['project_end_date', 'project_start_date'])

In [12]:
# Encode other_project_leader, 1 if other_project_leader, 0 if not applicable
ninds['ninds_other_project_leader'] = np.where(ninds[['other_project_leader']] == 'not applicable', 0, 1)
ninds = ninds.drop(columns='other_project_leader')

In [13]:
# Encode ninds_aff_name based on top 25 global universities for neuroscience and behavior, 1 if in top 25, 
# 0 if not in top 25, https://www.usnews.com/education/best-global-universities/neuroscience-behavior
top_neuro = ['harvard university', 'stanford university', 'university college london', 
             'university of california, san francisco', 'massachusetts institute of technology',
             'columbia university', 'university of oxford', 'university of pennsylvania', 
             'washinton university in st. louis', 'johns hopkins university', 
             'university of california, los angeles', 'university of california, san diego', 
             'university of cambridge', 'yale university', 'new york university', "king's college london", 
             'university of toronto', 'mcgill university', 'karolinska institute', 
             'university of california, berkeley', 'university of washington', 'mount sinai school of medicine', 
             'imperial college london', 'heidelberg university', 'university of pittsburgh']
ninds['ninds_aff_top_25_neuro'] = ninds['ninds_aff_name'].apply(lambda x: 0 if x not in top_neuro else 1)
ninds = ninds.drop(columns='ninds_aff_name')

In [14]:
# One hot encode ninds_aff_type
ninds = pd.concat([ninds, pd.get_dummies(ninds['ninds_aff_type'], prefix='ninds_aff_type')], 
                  axis=1).drop(['ninds_aff_type'], axis=1)
ninds.columns = ninds.columns.str.replace('  ', ' ')
ninds.columns = ninds.columns.str.replace(' ', '_')

In [15]:
# Drop ninds_aff_city
ninds = ninds.drop(columns='ninds_aff_city')

In [16]:
# Replace ninds_aff_state with U.S. region or international
northeast = ['ct', 'me', 'ma', 'nh', 'ri', 'vt', 'nj', 'ny', 'pa']
midwest = ['in', 'il', 'mi', 'oh', 'wi', 'ia', 'ks', 'mn', 'mo', 'ne', 'nd', 'sd']
south = ['de', 'fl', 'ga', 'md', 'nc', 'sc', 'va', 'wv', 'al', 'ky', 'ms', 'tn', 'ar', 'la', 'ok', 'tx']
west = ['mt', 'wy', 'co', 'nm', 'az', 'ut', 'id', 'nv', 'or', 'wa', 'ca', 'ak', 'hi']
ninds['ninds_aff_state'] = ninds['ninds_aff_state'].replace(northeast, 'northeast')
ninds['ninds_aff_state'] = ninds['ninds_aff_state'].replace(midwest, 'midwest')
ninds['ninds_aff_state'] = ninds['ninds_aff_state'].replace(south, 'south')
ninds['ninds_aff_state'] = ninds['ninds_aff_state'].replace(west, 'west')
ninds['ninds_aff_state'] = ninds['ninds_aff_state'].apply(lambda x: x if x in 
                                                          ['northeast', 'midwest', 'south', 'west'] 
                                                          else 'international')
# One hot encode ninds_aff_state
ninds = pd.concat([ninds, pd.get_dummies(ninds['ninds_aff_state'], prefix='ninds_aff')], 
                  axis=1).drop(['ninds_aff_state'], axis=1)
# Drop ninds_aff_international
ninds = ninds.drop(columns='ninds_aff_international')

In [17]:
# Encode ninds_aff_country, 1 if united states, 0 if international
ninds['ninds_aff_united_states'] = np.where(ninds[['ninds_aff_country']] == 'united states', 1, 0)
ninds = ninds.drop(columns='ninds_aff_country')

In [18]:
# Rename fiscal_year_total_cost
ninds = ninds.rename(columns={'fiscal_year_total_cost': 'ninds_fiscal_year_total_cost'})

In [19]:
# Check ninds columns
ninds.columns.to_list()

['ninds_support_year',
 'ninds_fiscal_year_total_cost',
 'scopus_idx',
 'ninds_grant_type_d43',
 'ninds_grant_type_dp1',
 'ninds_grant_type_dp2',
 'ninds_grant_type_f05',
 'ninds_grant_type_f06',
 'ninds_grant_type_f30',
 'ninds_grant_type_f31',
 'ninds_grant_type_f32',
 'ninds_grant_type_f33',
 'ninds_grant_type_f99',
 'ninds_grant_type_k01',
 'ninds_grant_type_k02',
 'ninds_grant_type_k04',
 'ninds_grant_type_k06',
 'ninds_grant_type_k08',
 'ninds_grant_type_k12',
 'ninds_grant_type_k17',
 'ninds_grant_type_k22',
 'ninds_grant_type_k23',
 'ninds_grant_type_k24',
 'ninds_grant_type_k25',
 'ninds_grant_type_k99',
 'ninds_grant_type_n01',
 'ninds_grant_type_n02',
 'ninds_grant_type_n43',
 'ninds_grant_type_n44',
 'ninds_grant_type_p01',
 'ninds_grant_type_p20',
 'ninds_grant_type_p30',
 'ninds_grant_type_p50',
 'ninds_grant_type_r00',
 'ninds_grant_type_r01',
 'ninds_grant_type_r03',
 'ninds_grant_type_r13',
 'ninds_grant_type_r15',
 'ninds_grant_type_r21',
 'ninds_grant_type_r24',
 'ni

In [20]:
# Check ninds
ninds.head()

,ninds_support_year,ninds_fiscal_year_total_cost,scopus_idx,ninds_grant_type_d43,ninds_grant_type_dp1,ninds_grant_type_dp2,ninds_grant_type_f05,ninds_grant_type_f06,ninds_grant_type_f30,ninds_grant_type_f31,...,ninds_aff_type_schools_of_osteopathy,ninds_aff_type_schools_of_pharmacy,ninds_aff_type_schools_of_public_health,ninds_aff_type_schools_of_veterinary_medicine,ninds_aff_type_unavailable,ninds_aff_midwest,ninds_aff_northeast,ninds_aff_south,ninds_aff_west,ninds_aff_united_states
0,14,885354.0,norman s abramson,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
4,13,75000.0,norman s abramson,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1
8,4,489233.0,ronald t acton,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
15,5,337995.0,ronald t acton,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,1
29,4,1776305.0,harold p adams,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1


In [21]:
# Drop NA
ninds = ninds.dropna()

In [22]:
# Get ninds shape
ninds.shape

(53582, 136)

## aff

In [23]:
# Encode aff_name based on top 25 global universities for neuroscience and behavior, 1 if in top 25, 
# 0 if not in top 25, https://www.usnews.com/education/best-global-universities/neuroscience-behavior
aff['aff_top_25_neuro'] = aff['aff_name'].apply(lambda x: 0 if x not in top_neuro else 1)
aff = aff.drop(columns='aff_name')

In [24]:
# Drop aff_city
aff = aff.drop(columns='aff_city')

In [25]:
# Replace aff_state with U.S. region or international
aff['aff_state'] = aff['aff_state'].replace(northeast, 'northeast')
aff['aff_state'] = aff['aff_state'].replace(midwest, 'midwest')
aff['aff_state'] = aff['aff_state'].replace(south, 'south')
aff['aff_state'] = aff['aff_state'].replace(west, 'west')
aff['aff_state'] = aff['aff_state'].apply(lambda x: x if x in 
                                          ['northeast', 'midwest', 'south', 'west'] 
                                          else 'international') 
# One hot encode aff_state
aff = pd.concat([aff, pd.get_dummies(aff['aff_state'], prefix='aff')], axis=1).drop(['aff_state'], axis=1)

In [26]:
# Encode ninds_aff_country, 1 if united states, 0 if international
aff['aff_united_states'] = np.where(aff[['aff_country']] == 'united states', 1, 0)
aff = aff.drop(columns='aff_country')

In [27]:
# Group by scopus_idx and sum to get total aff_top_25_neuro, aff_midwest, aff_northeast, aff_south, aff_west, 
# aff_international, aff_united_states
aff_total = aff.groupby('scopus_idx').sum()
aff_total = aff_total.reset_index()

In [28]:
# Group by scopus_idx and average to get average aff_author_count, aff_document_count
aff_avg = aff.groupby('scopus_idx').mean().round(0)
aff_avg = aff_avg.reset_index()

In [29]:
# Join aff_total and aff_avg
aff = pd.concat([aff_total.iloc[:, 3:], aff_avg.iloc[:, 0:3]], axis=1, join='inner')

In [30]:
# Rename columns
aff = aff.rename(columns={'aff_author_count': 'aff_avg_author_count', 
                          'aff_document_count': 'aff_avg_document_count', 
                          'aff_top_25_neuro': 'aff_total_top_25_neuro','aff_midwest': 'aff_total_midwest', 
                          'aff_northeast': 'aff_total_northeast', 'aff_south': 'aff_total_south', 
                          'aff_west': 'aff_total_west', 'aff_international': 'aff_total_international',
                          'aff_united_states': 'aff_total_united_states'})

In [31]:
# Check aff
aff.head()

,aff_total_top_25_neuro,aff_total_international,aff_total_midwest,aff_total_northeast,aff_total_south,aff_total_west,aff_total_united_states,scopus_idx,aff_avg_author_count,aff_avg_document_count
0,0,0,0,1,1,0,2,a dosemeci,2255.0,11868.0
1,1,6,2,0,1,0,3,a kulkarni,8215.0,40727.0
2,0,0,0,0,1,0,1,a lasansky,12448.0,210824.0
3,1,0,1,1,3,5,10,a wynshaw-boris,6755.0,54793.0
4,0,0,0,4,1,1,6,a a tzika,11950.0,81374.0


In [32]:
# Join ninds and aff
final_data = pd.merge(ninds, aff, left_on=['scopus_idx'], right_on=['scopus_idx'], how='inner')

In [33]:
# Check final_data columns
final_data.columns.to_list()

['ninds_support_year',
 'ninds_fiscal_year_total_cost',
 'scopus_idx',
 'ninds_grant_type_d43',
 'ninds_grant_type_dp1',
 'ninds_grant_type_dp2',
 'ninds_grant_type_f05',
 'ninds_grant_type_f06',
 'ninds_grant_type_f30',
 'ninds_grant_type_f31',
 'ninds_grant_type_f32',
 'ninds_grant_type_f33',
 'ninds_grant_type_f99',
 'ninds_grant_type_k01',
 'ninds_grant_type_k02',
 'ninds_grant_type_k04',
 'ninds_grant_type_k06',
 'ninds_grant_type_k08',
 'ninds_grant_type_k12',
 'ninds_grant_type_k17',
 'ninds_grant_type_k22',
 'ninds_grant_type_k23',
 'ninds_grant_type_k24',
 'ninds_grant_type_k25',
 'ninds_grant_type_k99',
 'ninds_grant_type_n01',
 'ninds_grant_type_n02',
 'ninds_grant_type_n43',
 'ninds_grant_type_n44',
 'ninds_grant_type_p01',
 'ninds_grant_type_p20',
 'ninds_grant_type_p30',
 'ninds_grant_type_p50',
 'ninds_grant_type_r00',
 'ninds_grant_type_r01',
 'ninds_grant_type_r03',
 'ninds_grant_type_r13',
 'ninds_grant_type_r15',
 'ninds_grant_type_r21',
 'ninds_grant_type_r24',
 'ni

In [34]:
# Check final_data
final_data.head()

,ninds_support_year,ninds_fiscal_year_total_cost,scopus_idx,ninds_grant_type_d43,ninds_grant_type_dp1,ninds_grant_type_dp2,ninds_grant_type_f05,ninds_grant_type_f06,ninds_grant_type_f30,ninds_grant_type_f31,...,ninds_aff_united_states,aff_total_top_25_neuro,aff_total_international,aff_total_midwest,aff_total_northeast,aff_total_south,aff_total_west,aff_total_united_states,aff_avg_author_count,aff_avg_document_count
0,14,885354.0,norman s abramson,0,0,0,0,0,0,0,...,1,1,0,0,4,1,0,5,7538.0,35914.0
1,13,75000.0,norman s abramson,0,0,0,0,0,0,0,...,1,1,0,0,4,1,0,5,7538.0,35914.0
2,4,489233.0,ronald t acton,0,0,0,0,0,0,0,...,1,0,2,0,0,9,1,11,5794.0,29254.0
3,5,337995.0,ronald t acton,0,0,0,0,0,0,0,...,1,0,2,0,0,9,1,11,5794.0,29254.0
4,4,1776305.0,harold p adams,0,0,0,0,0,0,0,...,1,0,1,4,1,1,0,7,4565.0,20706.0


In [35]:
# Drop NA
final_data = final_data.dropna()

In [36]:
# Get final_data shape
final_data.shape

(53582, 145)

## coauth

In [37]:
coauth.head()

,coauth_aff_name,coauth_aff_city,coauth_aff_country,scopus_idx
0,stanford university,palo alto,united states,jason c dugas
1,salk institute for biological studies,san diego,united states,jason c dugas
2,northwestern university feinberg school of med...,chicago,united states,jason c dugas
3,walter and eliza hall institute of medical res...,melbourne,australia,jason c dugas
4,"university of california, santa barbara",santa barbara,united states,jason c dugas


In [38]:
# Encode coauth_aff_name based on top 25 global universities for neuroscience and behavior, 1 if in top 25, 
# 0 if not in top 25, https://www.usnews.com/education/best-global-universities/neuroscience-behavior
coauth['coauth_aff_top_25_neuro'] = coauth['coauth_aff_name'].apply(lambda x: 0 if x not in top_neuro else 1)
coauth = coauth.drop(columns='coauth_aff_name')

In [39]:
# Drop coauth_aff_city
coauth = coauth.drop(columns='coauth_aff_city')

In [40]:
# Encode coauth_aff_country
coauth['coauth_aff_united_states'] = np.where(coauth[['coauth_aff_country']] == 'united states', 1, 0)
coauth['coauth_aff_international'] = np.where(coauth[['coauth_aff_country']] != 'united states', 1, 0)
coauth = coauth.drop(columns='coauth_aff_country')

In [41]:
# Group by scopus_idx and sum to get total coauth_aff_top_25_neuro and coauth_aff_united_states
coauth = coauth.groupby('scopus_idx').sum()
coauth = coauth.reset_index()

In [42]:
# Rename columns
coauth = coauth.rename(columns={'coauth_aff_top_25_neuro': 'coauth_total_aff_top_25_neuro',
                                'coauth_aff_united_states': 'coauth_total_aff_united_states',
                                'coauth_aff_international': 'coauth_total_aff_international'})

In [43]:
# Check coauth
coauth.head()

,scopus_idx,coauth_total_aff_top_25_neuro,coauth_total_aff_united_states,coauth_total_aff_international
0,a dosemeci,1,66,9
1,a kulkarni,21,52,108
2,a lasansky,1,4,6
3,a wynshaw-boris,18,104,56
4,a a tzika,16,125,29


In [44]:
# Join final_data and coauth
final_data = pd.merge(final_data, coauth, left_on=['scopus_idx'], right_on=['scopus_idx'], how='inner')

In [45]:
# Check final_data columns
final_data.columns.to_list()

['ninds_support_year',
 'ninds_fiscal_year_total_cost',
 'scopus_idx',
 'ninds_grant_type_d43',
 'ninds_grant_type_dp1',
 'ninds_grant_type_dp2',
 'ninds_grant_type_f05',
 'ninds_grant_type_f06',
 'ninds_grant_type_f30',
 'ninds_grant_type_f31',
 'ninds_grant_type_f32',
 'ninds_grant_type_f33',
 'ninds_grant_type_f99',
 'ninds_grant_type_k01',
 'ninds_grant_type_k02',
 'ninds_grant_type_k04',
 'ninds_grant_type_k06',
 'ninds_grant_type_k08',
 'ninds_grant_type_k12',
 'ninds_grant_type_k17',
 'ninds_grant_type_k22',
 'ninds_grant_type_k23',
 'ninds_grant_type_k24',
 'ninds_grant_type_k25',
 'ninds_grant_type_k99',
 'ninds_grant_type_n01',
 'ninds_grant_type_n02',
 'ninds_grant_type_n43',
 'ninds_grant_type_n44',
 'ninds_grant_type_p01',
 'ninds_grant_type_p20',
 'ninds_grant_type_p30',
 'ninds_grant_type_p50',
 'ninds_grant_type_r00',
 'ninds_grant_type_r01',
 'ninds_grant_type_r03',
 'ninds_grant_type_r13',
 'ninds_grant_type_r15',
 'ninds_grant_type_r21',
 'ninds_grant_type_r24',
 'ni

In [46]:
# Check final_data
final_data.head()

,ninds_support_year,ninds_fiscal_year_total_cost,scopus_idx,ninds_grant_type_d43,ninds_grant_type_dp1,ninds_grant_type_dp2,ninds_grant_type_f05,ninds_grant_type_f06,ninds_grant_type_f30,ninds_grant_type_f31,...,aff_total_midwest,aff_total_northeast,aff_total_south,aff_total_west,aff_total_united_states,aff_avg_author_count,aff_avg_document_count,coauth_total_aff_top_25_neuro,coauth_total_aff_united_states,coauth_total_aff_international
0,14,885354.0,norman s abramson,0,0,0,0,0,0,0,...,0,4,1,0,5,7538.0,35914.0,20,115,45
1,13,75000.0,norman s abramson,0,0,0,0,0,0,0,...,0,4,1,0,5,7538.0,35914.0,20,115,45
2,4,489233.0,ronald t acton,0,0,0,0,0,0,0,...,0,0,9,1,11,5794.0,29254.0,3,141,19
3,5,337995.0,ronald t acton,0,0,0,0,0,0,0,...,0,0,9,1,11,5794.0,29254.0,3,141,19
4,4,1776305.0,harold p adams,0,0,0,0,0,0,0,...,4,1,1,0,7,4565.0,20706.0,9,117,43


In [47]:
# Drop NA
final_data = final_data.dropna()

In [48]:
# Get final_data shape
final_data.shape

(53546, 148)

## scopus

In [49]:
# Rename support_year
scopus = scopus.rename(columns={'document_count': 'scopus_document_count', 
                                'coauthor_count': 'scopus_coauthor_count', 
                                'citation_count': 'scopus_ciation_count', 
                                'citedby_count': 'scopus_citedby_count',
                                'h_index': 'scopus_h_index'})

In [50]:
# Drop row with begin_publication_range equal to 1900
scopus = scopus[scopus['begin_publication_range'] != 1900]
# Get publication_range_years by subtracting begin_publication_range from end_publication_range 
scopus['scopus_pub_range_years'] = scopus['end_publication_range'] - scopus['begin_publication_range']
# Drop begin_publication_range and end_publication_range
scopus = scopus.drop(columns=['begin_publication_range', 'end_publication_range']) 

In [51]:
# Encode current_aff_name based on top 25 global universities for neuroscience and behavior, 1 if in top 25, 
# 0 if not in top 25, https://www.usnews.com/education/best-global-universities/neuroscience-behavior
scopus['scopus_current_aff_top_25_neuro'] = scopus['current_aff_name'].apply(lambda x: 0 if x not in top_neuro 
                                                                             else 1)
scopus = scopus.drop(columns='current_aff_name')

In [52]:
# Drop current_aff_city
scopus = scopus.drop(columns='current_aff_city')

In [53]:
# Replace current_aff_state with U.S. region or international
scopus['current_aff_state'] = scopus['current_aff_state'].replace(northeast, 'northeast')
scopus['current_aff_state'] = scopus['current_aff_state'].replace(midwest, 'midwest')
scopus['current_aff_state'] = scopus['current_aff_state'].replace(south, 'south')
scopus['current_aff_state'] = scopus['current_aff_state'].replace(west, 'west')
scopus['current_aff_state'] = scopus['current_aff_state'].apply(lambda x: x if x in 
                                                                ['northeast', 'midwest', 'south', 'west'] 
                                                                else 'international') 
# One hot encode current_aff_state
scopus = pd.concat([scopus, pd.get_dummies(scopus['current_aff_state'], prefix='scopus_current_aff')], 
                   axis=1).drop(['current_aff_state'], axis=1)
# Drop aff_international
scopus = scopus.drop(columns='scopus_current_aff_international')

In [54]:
# Encode ninds_aff_country, 1 if united states, 0 if international
scopus['scopus_current_aff_united_states'] = np.where(scopus[['current_aff_country']] == 'united states', 1, 0)
scopus = scopus.drop(columns='current_aff_country')

In [55]:
# Drop duplicate scopus_idx
scopus['scopus_idx'] = scopus['scopus_idx'].drop_duplicates(keep='first')

In [56]:
# Join final_data and scopus
final_data = pd.merge(final_data, scopus, left_on=['scopus_idx'], right_on=['scopus_idx'], how='inner')

In [57]:
# Check final_data columns
final_data.columns.to_list()

['ninds_support_year',
 'ninds_fiscal_year_total_cost',
 'scopus_idx',
 'ninds_grant_type_d43',
 'ninds_grant_type_dp1',
 'ninds_grant_type_dp2',
 'ninds_grant_type_f05',
 'ninds_grant_type_f06',
 'ninds_grant_type_f30',
 'ninds_grant_type_f31',
 'ninds_grant_type_f32',
 'ninds_grant_type_f33',
 'ninds_grant_type_f99',
 'ninds_grant_type_k01',
 'ninds_grant_type_k02',
 'ninds_grant_type_k04',
 'ninds_grant_type_k06',
 'ninds_grant_type_k08',
 'ninds_grant_type_k12',
 'ninds_grant_type_k17',
 'ninds_grant_type_k22',
 'ninds_grant_type_k23',
 'ninds_grant_type_k24',
 'ninds_grant_type_k25',
 'ninds_grant_type_k99',
 'ninds_grant_type_n01',
 'ninds_grant_type_n02',
 'ninds_grant_type_n43',
 'ninds_grant_type_n44',
 'ninds_grant_type_p01',
 'ninds_grant_type_p20',
 'ninds_grant_type_p30',
 'ninds_grant_type_p50',
 'ninds_grant_type_r00',
 'ninds_grant_type_r01',
 'ninds_grant_type_r03',
 'ninds_grant_type_r13',
 'ninds_grant_type_r15',
 'ninds_grant_type_r21',
 'ninds_grant_type_r24',
 'ni

In [58]:
# Check final_data
final_data.head()

,ninds_support_year,ninds_fiscal_year_total_cost,scopus_idx,ninds_grant_type_d43,ninds_grant_type_dp1,ninds_grant_type_dp2,ninds_grant_type_f05,ninds_grant_type_f06,ninds_grant_type_f30,ninds_grant_type_f31,...,scopus_ciation_count,scopus_citedby_count,scopus_h_index,scopus_pub_range_years,scopus_current_aff_top_25_neuro,scopus_current_aff_midwest,scopus_current_aff_northeast,scopus_current_aff_south,scopus_current_aff_west,scopus_current_aff_united_states
0,14,885354.0,norman s abramson,0,0,0,0,0,0,0,...,3875,3315,26,29,0,0,1,0,0,1
1,13,75000.0,norman s abramson,0,0,0,0,0,0,0,...,3875,3315,26,29,0,0,1,0,0,1
2,4,489233.0,ronald t acton,0,0,0,0,0,0,0,...,6194,4616,38,52,0,0,0,1,0,1
3,5,337995.0,ronald t acton,0,0,0,0,0,0,0,...,6194,4616,38,52,0,0,0,1,0,1
4,4,1776305.0,harold p adams,0,0,0,0,0,0,0,...,26063,21683,52,27,0,0,0,0,0,1


In [59]:
# Drop NA
final_data = final_data.dropna()

In [60]:
# Get final_data shape
final_data.shape

(53537, 160)

## jour

In [61]:
# Encode publication_name based on top 25 publications for biochemistry, genetics and molecular biology based on 
# H index, 1 if in top 25, 0 if not in top 25, https://www.scimagojr.com/journalrank.php?order=h&ord=desc&area=1300
top_biochem_gen_molec_pub = ['cell', 'journal of the americal chemical society', 'nature genetics', 
                             'nature medicine', 'journal of clinical oncology', 'journal of biological chemistry', 
                             'nucleic acids research', 'blood', 'cancer research', 'genes and development', 
                             'nature biotechnology', 'nature reviews cancer', 
                             'nature reviews molecular cell biology', 'embo journal', 'journal of cell biology', 
                             'molecular cell', 'nature cell biology', 'journal of the national cancer institute', 
                             'bioinformatics', 'biomaterials', 'journal of clinical endocrinology and metabolism', 
                             'oncohene', 'nature reviews genetics', 'physiological reviews', 
                             'molecular and cell biology']
jour['jour_pub_top_25_biochem_gen_molec'] = jour['publication_name'].apply(lambda x: 0 if x not in 
                                                                           top_biochem_gen_molec_pub else 1)

In [62]:
# Encode publication_name based on top 25 publications for neuroscience based on H index, 1 if in top 25, 
# 0 if not in top 25, https://www.scimagojr.com/journalrank.php?order=h&ord=desc&area=2800
top_neuro_pub = ['neuron', 'journal of neuroscience', 'nature neuroscience', 'embo journal', 
                 'nature reviews neuroscience', 'neuroimage', 'stroke', 'biological psychiatry', 
                 'annals of neurology', 'trends in neurosciences', 'trends in cognitive scienes', 
                 'pain', 'journal of neurophysiology', 'annual review of neuroscience', 'cerebral cortex', 
                 'annals of the new york academy of sciences', 'plos biology', 'journal of neurochemistry', 
                 'progress in neurobiology', 'current opinion in neurobiology', 
                 'neuroscience and biobehavioral reviews', 'neuroscience', 
                 'investigative ophthalmology and visual science', 'journal of cognitive neuroscience', 
                 'journal of comparative neurology']
jour['jour_pub_top_25_neuro'] = jour['publication_name'].apply(lambda x: 0 if x not in top_neuro_pub else 1)

In [63]:
# Encode publication_name based on top 25 publications for medicine based on H index, 1 if in top 25, 
# 0 if not in top 25, https://www.scimagojr.com/journalrank.php?area=2700&order=h&ord=desc
top_med_pub = ['new england journal of medicine', 'the lancet', 
               'jama - journal of the american medical association', 'circulation', 'nature medicine', 
               'journal of clinical oncology', 'journal of clinical investigation', 'blood', 'cancer research', 
               'journal of experimental medicine', 'journal of the american college of cardiology', 
               'british medical journal', 'nature reviews cancer', 'embo journal', 'gastroenterology', 
               'journal of physical chemistry b', 'annals of internal medicine', 'journal of cell biology', 
               'immunity', 'accounts of chemical research', 
               'american journal of respiratory and critical care medicine', 'nature reviews immunology', 
               'jama psychiatry', 'journal of the national cancer institute', 'neurology']
jour['jour_pub_top_25_med'] = jour['publication_name'].apply(lambda x: 0 if x not in top_med_pub else 1)

In [64]:
# Drop publication_name
jour = jour.drop(columns='publication_name')

In [65]:
# Drop cover_date 
jour = jour.drop(columns='cover_date')

In [66]:
# One hot encode publication_type
jour['publication_type'] = jour['publication_type'].replace("[{'$': 'journal'}, {'$': 'ar'}]", 'journal')
jour['publication_type'] = jour['publication_type'].replace("[{'$': '1353'}, {'$': 'journal'}]", 'journal')
jour = pd.concat([jour, pd.get_dummies(jour['publication_type'], prefix='jour_pub_type')], 
                 axis=1).drop(['publication_type'], axis=1)
jour.columns = jour.columns.str.replace(' ', '_')

In [67]:
# Drop open_access
jour = jour.drop(columns='open_access')

In [68]:
# Drop citedby_count
jour = jour.drop(columns='citedby_count')

In [69]:
# Group by scopus_idx and sum to get total pubs
jour_total = jour.groupby('scopus_idx').sum()
jour_total = jour_total.reset_index()

In [70]:
# Group by scopus_idx and average to get average author_count and citedby_count
jour_avg = jour.groupby('scopus_idx').mean().round(0)
jour_avg = jour_avg.reset_index()

In [74]:
# Join jour_total and jour_avg
jour = pd.concat([jour_total.iloc[:, 2:], jour_avg.iloc[:, 0:2]], axis=1, join='inner')

In [76]:
# Rename columns
jour = jour.rename(columns={'jour_pub_top_25_neuro': 'jour_total_pub_top_25_neuro',
                            'jour_pub_top_25_med': 'jour_total_pub_top_25_med',
                            'jour_pub_top_25_biochem_gen_molec': 'jour_total_pub_top_25_biochem_gen_molec', 
                            'jour_pub_type_book': 'jour_total_pub_type_book', 
                            'jour_pub_type_series': 'jour_total_pub_type_series', 
                            'jour_pub_type_proceeding': 'jour_total_pub_type_proceeding', 
                            'jour_pub_type_journal': 'jour_total_pub_type_journal', 
                            'jour_pub_type_report': 'jour_total_pub_type_report', 
                            'jour_pub_type_trade_journal': 'jour_total_pub_trade_journal',
                            'author_count': 'jour_avg_author_count'})

In [77]:
# Check jour
jour.head()

,jour_total_pub_top_25_biochem_gen_molec,jour_total_pub_top_25_neuro,jour_total_pub_top_25_med,jour_total_pub_type_book,jour_pub_type_book_series,jour_pub_type_conference_proceeding,jour_total_pub_type_journal,jour_total_pub_type_report,jour_total_pub_trade_journal,scopus_idx,jour_avg_author_count
0,2,19,0,0.0,0.0,0.0,60.0,0.0,0.0,a dosemeci,5.0
1,1,0,7,7.0,1.0,1.0,222.0,0.0,0.0,a kulkarni,8.0
2,1,0,1,0.0,1.0,0.0,29.0,0.0,0.0,a lasansky,2.0
3,56,24,20,3.0,9.0,0.0,210.0,0.0,0.0,a wynshaw-boris,10.0
4,0,3,0,1.0,1.0,3.0,68.0,0.0,0.0,a a tzika,10.0


In [78]:
# Join final_data and jour
final_data = pd.merge(final_data, jour, left_on=['scopus_idx'], right_on=['scopus_idx'], how='inner')

In [79]:
# Check final_data columns
final_data.columns.to_list()

['ninds_support_year',
 'ninds_fiscal_year_total_cost',
 'scopus_idx',
 'ninds_grant_type_d43',
 'ninds_grant_type_dp1',
 'ninds_grant_type_dp2',
 'ninds_grant_type_f05',
 'ninds_grant_type_f06',
 'ninds_grant_type_f30',
 'ninds_grant_type_f31',
 'ninds_grant_type_f32',
 'ninds_grant_type_f33',
 'ninds_grant_type_f99',
 'ninds_grant_type_k01',
 'ninds_grant_type_k02',
 'ninds_grant_type_k04',
 'ninds_grant_type_k06',
 'ninds_grant_type_k08',
 'ninds_grant_type_k12',
 'ninds_grant_type_k17',
 'ninds_grant_type_k22',
 'ninds_grant_type_k23',
 'ninds_grant_type_k24',
 'ninds_grant_type_k25',
 'ninds_grant_type_k99',
 'ninds_grant_type_n01',
 'ninds_grant_type_n02',
 'ninds_grant_type_n43',
 'ninds_grant_type_n44',
 'ninds_grant_type_p01',
 'ninds_grant_type_p20',
 'ninds_grant_type_p30',
 'ninds_grant_type_p50',
 'ninds_grant_type_r00',
 'ninds_grant_type_r01',
 'ninds_grant_type_r03',
 'ninds_grant_type_r13',
 'ninds_grant_type_r15',
 'ninds_grant_type_r21',
 'ninds_grant_type_r24',
 'ni

In [80]:
# Check final_data
final_data.head()

,ninds_support_year,ninds_fiscal_year_total_cost,scopus_idx,ninds_grant_type_d43,ninds_grant_type_dp1,ninds_grant_type_dp2,ninds_grant_type_f05,ninds_grant_type_f06,ninds_grant_type_f30,ninds_grant_type_f31,...,jour_total_pub_top_25_biochem_gen_molec,jour_total_pub_top_25_neuro,jour_total_pub_top_25_med,jour_total_pub_type_book,jour_pub_type_book_series,jour_pub_type_conference_proceeding,jour_total_pub_type_journal,jour_total_pub_type_report,jour_total_pub_trade_journal,jour_avg_author_count
0,14,885354.0,norman s abramson,0,0,0,0,0,0,0,...,0,0,3,0.0,0.0,0.0,70.0,0.0,0.0,9.0
1,13,75000.0,norman s abramson,0,0,0,0,0,0,0,...,0,0,3,0.0,0.0,0.0,70.0,0.0,0.0,9.0
2,4,489233.0,ronald t acton,0,0,0,0,0,0,0,...,6,0,9,0.0,2.0,0.0,246.0,0.0,0.0,9.0
3,5,337995.0,ronald t acton,0,0,0,0,0,0,0,...,6,0,9,0.0,2.0,0.0,246.0,0.0,0.0,9.0
4,4,1776305.0,harold p adams,0,0,0,0,0,0,0,...,0,46,31,13.0,5.0,0.0,186.0,0.0,0.0,8.0


In [81]:
# Drop NA
final_data = final_data.dropna()

In [82]:
# Get final_data shape
final_data.shape

(53485, 170)

## final_data

In [83]:
# Out final_data as csv
final_data.to_csv('data/final_data.csv')